### mart 판매 데이터를 기반으로 판매액을 예측하시오.
- 제공된 데이터 목록: mart_train.csv (훈련 데이터), mart_test.csv (평가용 데이터)
- 예측할 컬럼: total (총 판매액)
학습용 데이터(mart_train.csv)를 이용하여 총 판매액을 예측하는 모델을 만든 후, 이를 평가용 데이터(mart_test.csv)에 적용하여 얻은 예측값을 다음과 같은 형식의 CSV 파일로 생성하시오.
- 제출 파일은 다음 한 개의 컬럼을 포함해야 합니다.
- pred: 예측된 총 판매액
- 제출 파일명: 'result.csv'
- 제출한 모델의 성능은 RMSE(Root Mean Square Error) 평가지표에 따라 채점한다.
- 제출 CSV 파일명 및 형태: result.csv

~~~
pred
10000
20000
30000
40000
...
~~~

### 답안 제출 참고
- pd.read_csv('result.csv') 로 제출 코드 확인

# 1. 문제정의
- RMSE
- target: total
- 예측 파일명: result.csv
- 컬럼 1개(pred)

# 2. 라이브러리 및 데이터 불러오기

In [72]:
# 데이터 불러오기
import pandas as pd
train = pd.read_csv("mart_train.csv")
test = pd.read_csv("mart_test.csv")

# 3. 탐색적 데이터 분석(EDA)

In [73]:
# 데이터 크기 확인
print(train.shape, test.shape)
#train에는 total 존재. test에는 부재.

(700, 10) (300, 9)


In [74]:
# train 샘플 확인
print(train.info()) # 수치형 데이터는 2개. 나머지는 범주형.
print(train.head())
#범주형이 많으므로 인코딩 필요할 것으로 보임

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   branch          700 non-null    object 
 1   city            700 non-null    object 
 2   customer_type   700 non-null    object 
 3   gender          700 non-null    object 
 4   product_line    700 non-null    object 
 5   total           700 non-null    float64
 6   payment_method  700 non-null    object 
 7   rating          700 non-null    float64
 8   time_of_day     700 non-null    object 
 9   day_name        700 non-null    object 
dtypes: float64(2), object(8)
memory usage: 54.8+ KB
None
  branch       city customer_type  gender            product_line      total  \
0      A     Yangon        Member  Female       Health and beauty  823457.25   
1      C  Naypyitaw        Normal  Female  Electronic accessories  120330.00   
2      A     Yangon        Normal    Male      Home and l

In [75]:
# test 샘플 확인
print(test.info())
print(test.head())
# 수치형 1개

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   branch          300 non-null    object 
 1   city            300 non-null    object 
 2   customer_type   300 non-null    object 
 3   gender          300 non-null    object 
 4   product_line    300 non-null    object 
 5   payment_method  300 non-null    object 
 6   rating          300 non-null    float64
 7   time_of_day     300 non-null    object 
 8   day_name        300 non-null    object 
dtypes: float64(1), object(8)
memory usage: 21.2+ KB
None
  branch       city customer_type  gender         product_line payment_method  \
0      C  Naypyitaw        Normal  Female  Fashion accessories        Ewallet   
1      B   Mandalay        Normal    Male   Food and beverages    Credit card   
2      B   Mandalay        Member  Female  Fashion accessories    Credit card   
3      B   Mandalay

In [76]:
# 자료형 확인
print(train.describe(include = 'O'))
print(test.describe(include = 'O'))



       branch    city customer_type gender       product_line payment_method  \
count     700     700           700    700                700            700   
unique      3       3             2      2                  6              3   
top         A  Yangon        Normal   Male  Sports and travel           Cash   
freq      236     236           354    356                127            246   

       time_of_day  day_name  
count          700       700  
unique           3         7  
top        evening  Saturday  
freq           309       114  
       branch    city customer_type  gender            product_line  \
count     300     300           300     300                     300   
unique      3       3             2       2                       6   
top         A  Yangon        Member  Female  Electronic accessories   
freq      104     104           155     157                      58   

       payment_method time_of_day  day_name  
count             300         300       30

In [77]:
# train 기초 통계값 확인
train.describe()

,total,rating
count,7.000000e+02,700.000000
mean,4.850780e+05,7.003429
std,3.643907e+05,1.713078
min,1.904175e+04,4.000000
25%,2.001195e+05,5.500000
50%,3.818745e+05,7.000000
75%,7.061276e+05,8.425000
max,1.563975e+06,10.000000


In [78]:
# test 기초 통계값 확인
test.describe()

,rating
count,300.000000
mean,6.901000
std,1.732099
min,4.000000
25%,5.500000
50%,6.800000
75%,8.500000
max,10.000000


In [79]:
# train 기초 통계값 (Object)
train.describe(include = 'O')

,branch,city,customer_type,gender,product_line,payment_method,time_of_day,day_name
count,700,700,700,700,700,700,700,700
unique,3,3,2,2,6,3,3,7
top,A,Yangon,Normal,Male,Sports and travel,Cash,evening,Saturday
freq,236,236,354,356,127,246,309,114


In [80]:
# test 기초 통계값 (Object)


In [81]:
# train 결측치
print(train.isnull().sum()) # train 결측치 없음.

branch            0
city              0
customer_type     0
gender            0
product_line      0
total             0
payment_method    0
rating            0
time_of_day       0
day_name          0
dtype: int64


In [82]:
# test 결측치
print(test.isnull().sum())

branch            0
city              0
customer_type     0
gender            0
product_line      0
payment_method    0
rating            0
time_of_day       0
day_name          0
dtype: int64


In [83]:
# 타겟 (value_count())
print(train.value_counts())

branch  city       customer_type  gender  product_line            total       payment_method  rating  time_of_day  day_name
A       Yangon     Member         Female  Electronic accessories  98406.00    Cash            4.7     evening      Monday      1
C       Naypyitaw  Member         Female  Electronic accessories  733477.50   Cash            7.6     afternoon    Sunday      1
B       Mandalay   Normal         Male    Sports and travel       541327.50   Ewallet         6.7     morning      Saturday    1
                                                                  615762.00   Ewallet         6.4     evening      Tuesday     1
                                                                  817582.50   Cash            8.2     afternoon    Saturday    1
                                                                                                                              ..
A       Yangon     Normal         Male    Sports and travel       1240627.50  Cash            6.4     

In [84]:
# target (분포)
target = train['total']
print(target.value_counts())

total
283641.75     2
263875.50     2
415422.00     2
326450.25     2
130851.00     2
             ..
293391.00     1
137103.75     1
348232.50     1
104107.50     1
1535625.00    1
Name: count, Length: 695, dtype: int64


In [85]:
# target (분포, 시각화)
#pd.set_option('display.float_format,' '{:.10f}'.format)
#train['total'].describe()

# 4. 데이터 전처리

In [86]:
# target
target = train.pop('total')
print(target) # train데이터의 total을 target으로 옮김.

0       823457.25
1       120330.00
2       510788.25
3       733572.00
4       951567.75
          ...    
695     688038.75
696     170352.00
697     391797.00
698     986296.50
699    1535625.00
Name: total, Length: 700, dtype: float64


In [87]:
# 원핫인코딩
train = pd.get_dummies(train)
test = pd.get_dummies(test)

train.shape, test.shape # 컬럼 수 통일.

((700, 30), (300, 30))

# 5. 검증 데이터 분할

In [88]:
# 검증데이터 분할
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size = 0.15, random_state = 2024)

X_tr.shape, X_val.shape, y_tr.shape, y_val.shape

((595, 30), (105, 30), (595,), (105,))

# 6. 머신러닝 학습 및 평가

In [89]:
# RMSE
from sklearn.metrics import mean_squared_error

def rmse(a,b):
  return (mean_squared_error(a,b) ** 0.5)

In [90]:
# 선형회귀
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_tr, y_tr)
pred = lr.predict(X_val)
rmse(y_val, pred)

398188.9222552941

In [67]:
# 랜덤포레스트
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor()
rf.fit(X_tr, y_tr)
# print(dir(RandomForestRegressor))
pred = rf.predict(X_val)
# print(pred)
print(rmse(y_val, pred))
# 검증 데이터에 대하여 채점.



411262.89269079757


In [68]:
# Xgboost
import xgboost as xgb
xg = xgb.XGBRegressor(random_state = 2024)
xg.fit(X_tr, y_tr)
pred = xg.predict(X_val)
rmse(y_val, pred)

470162.398813129

In [69]:
from re import X
# LightGBM
import lightgbm as lgb
lg = lgb.LGBMRegressor(random_state = 2024, verbose = -1)
lg.fit(X_tr, y_tr)
pred = lg.predict(X_val)
rmse(y_val, pred)


427879.09123489517

# 7. 예측 및 결과 파일 생성

In [98]:
# test예측 및 csv 생성
pred = lr.predict(test)
print(len((pred)))

result = pd.DataFrame({
    'pred' : pred

})

print(result.shape)
#result.to_csv('result.csv', index = False)






300
(300, 1)


In [99]:
# result.csv 확인
result = pd.DataFrame({
    'pred': pred
})

result.to_csv('result.csv', index = False)

In [100]:
df = pd.read_csv('result.csv')
print(df)

              pred
0    458508.257360
1    449544.720829
2    419567.748598
3    516669.884280
4    560995.317570
..             ...
295  510337.506173
296  537135.598847
297  566095.209861
298  537799.830481
299  483296.357922

[300 rows x 1 columns]


# [심화] 성능 개선

In [116]:
# 데이터 불러오기
import pandas as pd
train = pd.read_csv("mart_train.csv")
test = pd.read_csv("mart_test.csv")

# target 데이터
target = train.pop('total')
print(target)

# 레이블 인코딩
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cols = train.select_dtypes(include = 'object').columns
for col in cols:
  train[col] = le.fit_transform(train[col])
  test[col] = le.fit_transform(test[col])

print(train.shape, test.shape)

# 검증데이터 분리
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size = 0.2, random_state = 2024)
X_tr.shape, X_val.shape, y_tr.shape, y_val.shape

# 선형회귀
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_tr, y_tr)
pred = lr.predict(X_val)
#print(pred)
# 랜덤포레스트
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_tr, y_tr)
rfPred = rf.predict(X_val)
# print(rfPred)

# Xgboost
import xgboost as xgb

xg = xgb.XGBRegressor(random_state = 2024)
xg.fit(X_tr, y_tr)
xgPred = xg.predict(X_val)
#print(xgPred)
# LightGBM

import lightgbm as lgb
lg = lgb.LGBMRegressor(random_state = 2024, verbose = -1)
lg.fit(X_tr, y_tr)
lgPred = lg.predict(X_val)
print(lgPred)


# 최종 제출 파일

pred = lr.predict(test)
result = pd.DataFrame({
    'pred': pred
})

#print(result.shape)
result.to_csv('result2.csv', index = False)


0       823457.25
1       120330.00
2       510788.25
3       733572.00
4       951567.75
          ...    
695     688038.75
696     170352.00
697     391797.00
698     986296.50
699    1535625.00
Name: total, Length: 700, dtype: float64
(700, 9) (300, 9)
[326726.78250025 634177.24935868 437956.8605004  753426.14546899
 155997.64370213 511492.71492921 478750.25591485 632354.95433572
 699959.78802031 562047.4954148  536677.28946667 527704.1236099
 372150.04191378 289748.2904052  325538.47729634 520046.02108303
 181650.4737829  511004.6037184  541337.10254703 539305.60988531
 674482.75503109 406144.35941091 371032.43759724 602967.73897047
 362089.40429939 571846.58119242 727600.38175628 269747.06790663
 510220.63757038 440194.21899611 203476.74764631 369369.16902342
 244536.06725844 641932.90399658 569997.57566165 527556.00015608
 557117.63432041 685345.52372473 648756.70808744 725009.98752733
  46027.63269892 186656.10356773 514694.97559484 530809.49630926
 570930.78211632 721229.06380

ValueError: Found input variables with inconsistent numbers of samples: [140, 300]